# Keras: deep dive

Different ways to build Keras models
There are three APIs for building models in Keras:

1. The Sequential model, the most approachable API—it’s basically a Python list. As such, it’s limited to simple stacks of layers.
2. The Functional API, which focuses on graph-like model architectures. It rep- resents a nice mid-point between usability and flexibility, and as such, it’s the most commonly used model-building API.
3. Model subclassing, a low-level option where you write everything yourself from scratch. This is ideal if you want full control over every little thing. However, you won’t get access to many built-in Keras features, and you will be more at risk of making mistakes.

## The sequential model
The simplest way to build a Keras model is to use the Sequential model, which you already know about.

In this approach we simply pass the netwrok structur as a python list:

In [ ]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(10)
])

It is also possible to use the add method to create the same model:

In [ ]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Dense(64))
model2.add(tf.keras.layers.Dense(10))

In the following code we use the summary method to print both models. However, we first need to Build the model first by calling `build()` or by calling the model on a batch of data:

In [ ]:
model1.build(input_shape=(None,3))
model2.build(input_shape=(None,3))

You can see that both models are equal, the only thing that differs is the name of the model:

In [ ]:
print(model1.summary())
print(model2.summary())

Despite, the weights are different because the model is always initialized with random weights.

In [ ]:
print(model1.weights[0][1])
print(model2.weights[0][1])

We can also change the name of the model

In [ ]:
model = tf.keras.Sequential(name="example_model")
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dense(10))
model.build(input_shape=(None,3))
model.summary()

Sometimes you need to get the summary of the model when you add the layers incrementally. In this scenario, you can set the input shape for the model so you do not need to build the model as it gets built on the fly:

In [ ]:
model = tf.keras.Sequential(name="test_model")
model.add(tf.keras.Input(shape=(None,3)))
model.add(tf.keras.layers.Dense(10))
model.summary()

## The functional API

The Sequential model is easy to use, but its applicability is extremely limited: it can only express models with a single input and a single output, applying one layer after the other in a sequential fashion. In practice, it’s pretty common to encounter models with multiple inputs (say, an image and its metadata), multiple outputs (different things you want to predict about the data), or a nonlinear topology.

In such cases, you’d build your model using the Functional API. This is what most Keras models you’ll encounter in the wild use. It’s fun and powerful. it feels like playing with LEGO bricks.

First, we create a single inpu, single output model similar to that last one we created to understand the functional API:

In [ ]:
inputs = tf.keras.Input(shape=(3,))
features = tf.keras.layers.Dense(64)(inputs)
outputs = tf.keras.layers.Dense(10)(features)
model = tf.keras.Model(inputs=inputs , outputs=outputs)
model.summary()

Now, we use this functional API to create a multi input, multi output model:

In [ ]:
import pandas as pd
df = pd.read_csv("./files/AllThrillerSeriesList.csv")
df

In [ ]:
def convert(str):
    return float(str)
ratings = df["Ratings"].apply(convert)

In [ ]:
x = df["Year"].apply(lambda value: value.split("–"))

In [ ]:
start_year = []
years = []
for index, item in enumerate(x):
    try:
        start_year.append(int(item[0]))
    except:
        start_year.append(2000)
    if len(item)==2:
        try:
            years.append(int(item[1]) - int(item[0]))
        except:
            years.append(0)    
    else:
        years.append(0)

In [ ]:
df["Votes"] = df["Votes"].apply(lambda x: int(x.replace(",","")))

In [ ]:
mean = df["Votes"].mean()
print(mean)
popularity_labels = []
for v in df["Votes"]:
    if v > mean:
        popularity_labels.append(1)
    else:
        popularity_labels.append(0)


In [ ]:
def create_dict(data):
    values = data.apply(lambda g: g.split(","))
    dictionary = {}
    index = 0
    for item in values:
        for g in item:
            g = g.strip()
            if dictionary.get(g, None) == None:
                dictionary[g] = index
                index = index + 1
    return dictionary

In [ ]:
genres_dict = create_dict(df["Genre"])
genres_dict

In [ ]:
def one_hot(data, dictionary, split = ","):
    rows = data.apply(lambda g: g.split(split))
    output = np.zeros([len(rows), len(dictionary)])
    for index, row in enumerate(rows):
        for key in row:
            key = key.strip()
            if dictionary.get(key, None) != None:
                output[index,dictionary[key]] = 1
    return output

In [ ]:
genres_encoded = one_hot(df["Genre"],genres_dict,",")
print(genres_encoded.shape)
genres_encoded

In [ ]:
actors_dict = create_dict(df["Actor/Actress"])
actors_dict

In [ ]:
actors_encoded = one_hot(df["Actor/Actress"], actors_dict , ",")
print(actors_encoded.shape)
actors_encoded

In [ ]:
year_duration = tf.keras.Input(shape=(2), name="year_duration")
genre = tf.keras.Input(shape=(24,), name="genre")
actors = tf.keras.Input(shape=(10827,), name="actors")

features = tf.keras.layers.Concatenate()([year_duration , genre , actors])
features = tf.keras.layers.Dense(256,activation="relu")(features)
features = tf.keras.layers.Dense(256,activation="relu")(features)
features = tf.keras.layers.Dense(64,activation="relu")(features)

popularity = tf.keras.layers.Dense(1, activation="softmax" , name="popularity")(features)
rating = tf.keras.layers.Dense(1, activation="sigmoid" , name="rating")(features)

model = tf.keras.Model(inputs = [year_duration , genre , actors] , outputs=[popularity , rating])


In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
model.compile(optimizer="rmsprop", loss={"rating": "mean_squared_error" , "popularity" : "categorical_crossentropy"}
 ,metrics={"rating": ["mean_absolute_error"], "popularity": ["accuracy"]})

In [ ]:
model.fit({"year_duration": np.array([start_year , years]).T , "genre": genres_encoded,
           "actors": actors_encoded},
          {"popularity": np.array(popularity_labels), "rating": np.array(ratings)},
          epochs=10)

## Subclassing the Model class

## Custom metrics



## Using callbacks
Here are some examples of ways you can use callbacks:
* Model checkpointing—Saving the current state of the model at different points during training.
* Early stopping—Interrupting training when the validation loss is no longer improving (and of course, saving the best model obtained during training).
* Dynamically adjusting the value of certain parameters during training—Such as the learning rate of the optimizer.
* Logging training and validation metrics during training, or visualizing the representa- tions learned by the model as they’re updated—The fit() progress bar that you’re familiar with is in fact a callback!

### Builtin callbacks

keras.callbacks.ModelCheckpoint
keras.callbacks.EarlyStopping
keras.callbacks.LearningRateScheduler
keras.callbacks.ReduceLROnPlateau
keras.callbacks.CSVLogger

### custom callbacks

Callbacks are implemented by sub- classing the keras.callbacks.Callback